## ESS Data Usage Example - MTMount accelerometers
Querying MTMount accelerometers \
Craig Lage - 08-Sep-25

In [ ]:
import asyncio
import matplotlib.pyplot as plt 
import numpy as np
import pandas as pd
import os
import sys
import time
import warnings
from lsst.daf.butler import Butler
from lsst.summit.utils.butlerUtils import getExpRecordFromDataId
from astropy.time import Time, TimeDelta
from lsst_efd_client.efd_helper import merge_packed_time_series
from lsst.summit.utils.efdUtils import makeEfdClient, getEfdData
# Ignore the many warning messages from ``merge_packed_time_series``
warnings.simplefilter(action="ignore", category=FutureWarning)

In [ ]:
# Get EFD client
client = makeEfdClient()
butler = Butler('/repo/embargo', collections=['LSSTCam/raw/all', 
                                            'LSSTCam/calib/unbounded', 'LSSTCam/runs/nightlyValidation'])

In [ ]:
expId = 2025082800505
dataId = {'exposure':expId, 'instrument':'LSSTCam'}
expRecord = getExpRecordFromDataId(butler, dataId)
start = expRecord.timespan.begin.utc
end = expRecord.timespan.end.utc

In [ ]:
baseFields = ["accelerationX", "accelerationY", "accelerationZ"]
sensorNames = [
    "SST top end ring +x -y",
    "SST top end ring -x -y",
    "SST spider spindle"
]

##  Get all of the data for the selected times

In [ ]:
packed_dataframe = await client.select_time_series(
    "lsst.sal.ESS.accelerometer", ["*"], start, end
)

## Now unpack it and plot it by sensor

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(8, 8))
plt.subplots_adjust(hspace=0.5, wspace=0.5)
for i, sensorName in enumerate(sensorNames):
    plotx = i % 2
    ploty = int(i / 2)
    ax = axs[plotx][ploty]
    if i > 2:
        ax.axes('off')
        continue

    sub_dataframe = packed_dataframe.loc[packed_dataframe.sensorName == sensorName]
    ax.set_title(sensorName)
    ax.set_ylabel("Accel(m/s^2)")
    for baseField in baseFields:
        df = merge_packed_time_series(
            sub_dataframe,
            baseField,
            stride=1,
            ref_timestamp_col="timestamp",
            fmt="unix_tai",
            scale="tai",
        )
        df[baseField].plot(ax=ax, label=list(baseField)[-1])
    ax.legend()
plt.savefig("/home/c/cslage/u/MTMount/vel_accel_jerk_plots/TMA_Accels_2025082500505.png")